In [1]:
import multiprocessing
import shutil

import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics

from autosklearn.metrics import accuracy
from autosklearn.classification import AutoSklearnClassifier
from autosklearn.constants import *

In [2]:
tmp_folder = '/tmp/autosklearn_parallel_example_tmp'
output_folder = '/tmp/autosklearn_parallel_example_out'


for dir in [tmp_folder, output_folder]:
    try:
        shutil.rmtree(dir)
    except OSError as e:
        pass

In [3]:
def spawn_classifier(seed, dataset_name):
    """Spawn a subprocess.
    auto-sklearn does not take care of spawning worker processes. This
    function, which is called several times in the main block is a new
    process which runs one instance of auto-sklearn.
    """

    # Use the initial configurations from meta-learning only in one out of
    # the four processes spawned. This prevents auto-sklearn from evaluating
    # the same configurations in four processes.
    if seed == 0:
        initial_configurations_via_metalearning = 25
    else:
        initial_configurations_via_metalearning = 0

    # Arguments which are different to other runs of auto-sklearn:
    # 1. all classifiers write to the same output directory
    # 2. shared_mode is set to True, this enables sharing of data between
    # models.
    # 3. all instances of the AutoSklearnClassifier must have a different seed!
    automl = AutoSklearnClassifier(
        time_left_for_this_task=60, # sec., how long should this seed fit
        # process run
        per_run_time_limit=15, # sec., each model may only take this long before it's killed
        ml_memory_limit=1024, # MB, memory limit imposed on each call to a ML algorithm
        shared_mode=True, # tmp folder will be shared between seeds
        tmp_folder=tmp_folder,
        output_folder=output_folder,
        delete_tmp_folder_after_terminate=False,
        ensemble_size=0, # ensembles will be built when all optimization runs are finished
        initial_configurations_via_metalearning=initial_configurations_via_metalearning,
        seed=seed)
    automl.fit(X_train, y_train, dataset_name=dataset_name)

In [4]:
X, y = sklearn.datasets.load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = \
    sklearn.model_selection.train_test_split(X, y, random_state=1)

processes = []
for i in range(4): # set this at roughly half of your cores
    p = multiprocessing.Process(target=spawn_classifier, args=(i, 'digits'))
    p.start()
    processes.append(p)
for p in processes:
    p.join()

print('Starting to build an ensemble!')
automl = AutoSklearnClassifier(time_left_for_this_task=15,
                               per_run_time_limit=15,
                               ml_memory_limit=1024,
                               shared_mode=True,
                               ensemble_size=50,
                               ensemble_nbest=200,
                               tmp_folder=tmp_folder,
                               output_folder=output_folder,
                               initial_configurations_via_metalearning=0,
                               seed=1)

# Both the ensemble_size and ensemble_nbest parameters can be changed now if
# necessary
automl.fit_ensemble(y_train,
                    task=MULTICLASS_CLASSIFICATION,
                    metric=accuracy,
                    precision='32',
                    dataset_name='digits',
                    ensemble_size=20,
                    ensemble_nbest=50)

predictions = automl.predict(X_test)
print(automl.show_models())
print("Accuracy score", sklearn.metrics.accuracy_score(y_test, predictions))

Starting to build an ensemble!
[(0.950000, MyDummyClassifier(configuration=1, init_params=None, random_state=None)),
(0.050000, SimpleClassificationPipeline({'classifier:liblinear_svc:C': 9901.884556489502, 'preprocessor:polynomial:interaction_only': 'False', 'one_hot_encoding:use_minimum_fraction': 'True', 'classifier:liblinear_svc:intercept_scaling': 1, 'rescaling:__choice__': 'minmax', 'preprocessor:__choice__': 'polynomial', 'classifier:liblinear_svc:multi_class': 'ovr', 'classifier:liblinear_svc:loss': 'squared_hinge', 'imputation:strategy': 'mean', 'preprocessor:polynomial:include_bias': 'False', 'classifier:liblinear_svc:fit_intercept': 'True', 'one_hot_encoding:minimum_fraction': 0.010000000000000004, 'balancing:strategy': 'none', 'classifier:__choice__': 'liblinear_svc', 'preprocessor:polynomial:degree': 2, 'classifier:liblinear_svc:penalty': 'l2', 'classifier:liblinear_svc:tol': 0.0023772047112092503, 'classifier:liblinear_svc:dual': 'False'},
dataset_properties={
  'signed':